In [1]:
# !pip install tensorflow==1.14.0
import tensorflow as tf
print(tf.__version__)

C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-packages\tensorflow\pyth

1.14.0


C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jimmy\anaconda3\envs\TF 114\lib\site-pac

In [2]:
from __future__ import division, print_function, unicode_literals
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os, os.path
import cv2
import random

In [3]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [4]:
np.random.seed(42)

In [5]:
def image_noaugmentation(imagesArray,imagesLabels,path):
    full_path = path+"live\\"
    ImagesPaths = [x for x in os.listdir(full_path)]
    
    count = 0
    for i in ImagesPaths:
        img = cv2.imread(full_path+i)
        img = cv2.resize(img,(img_width,img_height))
        imagesArray[count]= img
        imagesLabels[count]=np.array(0)
        count+=1
        
    full_path = path+"spoof\\"
    ImagesPaths = [x for x in os.listdir(full_path)]
    
    for i in ImagesPaths:
        img = cv2.imread(full_path+i)
        img = cv2.resize(img,(img_width,img_height))
        imagesArray[count]= img
        imagesLabels[count]=np.array(1)
        count+=1

In [6]:
from tensorflow.keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives)
        return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives)
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [7]:
K.epsilon()

1e-07

In [8]:
img_width, img_height, img_num_channels = 50, 50, 3
#search about face Pixels


input_shape = (img_width, img_height, img_num_channels)
pathTrain = 'E:\\Python36-32\\Face-Recognition-Liveness-Detection\\CelebA_Spoof\\Data\\train\\'
imagesArrayTrain = np.zeros((2102,img_width,img_height,img_num_channels),dtype = 'float32')
imagesLabelsTrain = np.zeros((2102,1),dtype = 'uint8')

In [9]:
pathTest = 'E:\\Python36-32\\Face-Recognition-Liveness-Detection\\CelebA_Spoof\\Data\\test\\'
imagesArrayTest = np.zeros((196,img_width,img_height,img_num_channels),dtype = 'float32')
imagesLabelsTest = np.zeros((196,1),dtype = 'uint8')

In [10]:
image_noaugmentation(imagesArrayTrain,imagesLabelsTrain,pathTrain)
imagesArrayTrain = imagesArrayTrain / 255
shuffler = np.random.permutation(len(imagesArrayTrain))
imagesArrayTrain_shuffled = imagesArrayTrain[shuffler]
imagesLabelsTrain_shuffled = imagesLabelsTrain[shuffler]

In [11]:
image_noaugmentation(imagesArrayTest,imagesLabelsTest,pathTest)
imagesArrayTest = imagesArrayTest / 255
shuffler = np.random.permutation(len(imagesArrayTest))
X_test = imagesArrayTest[shuffler]
y_test = imagesLabelsTest[shuffler]

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(imagesArrayTrain_shuffled, imagesLabelsTrain_shuffled,stratify=imagesLabelsTrain_shuffled, test_size=0.2)

In [13]:
X = tf.placeholder(shape=[None, 50, 50, 3], dtype=tf.float32, name="X")

In [14]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 17 * 17  # 1152 primary capsules
caps1_n_dims = 8

In [15]:
conv1_params = {
    "filters": 256,
    "kernel_size": 9,
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
    "kernel_size": 9,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu
}

In [16]:
conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)
conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")

In [18]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

In [19]:
caps1_output = squash(caps1_raw, name="caps1_output")


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [20]:
caps2_n_caps = 10
caps2_n_dims = 16

In [21]:
init_sigma = 0.1

W_init = tf.random_normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

In [22]:
batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

In [23]:
caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")

In [24]:
caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")

In [25]:
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")

In [26]:
routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")


Instructions for updating:
dim is deprecated, use axis instead


In [27]:
weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                             name="weighted_sum")

In [28]:
caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")

In [29]:
caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

In [30]:
agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")

In [31]:
raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")

In [32]:
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")


In [33]:
caps2_output = caps2_output_round_2


In [34]:
def condition(input, counter):
    return tf.less(counter, 100)

def loop_body(input, counter):
    output = tf.add(input, tf.square(counter))
    return output, tf.add(counter, 1)

with tf.name_scope("compute_sum_of_squares"):
    counter = tf.constant(1)
    sum_of_squares = tf.constant(0)

    result = tf.while_loop(condition, loop_body, [sum_of_squares, counter])
    

with tf.Session() as sess:
    print(sess.run(result))


(328350, 100)


In [35]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)


In [36]:
y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")


In [37]:
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")


In [38]:
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
y_pred

<tf.Tensor 'y_pred:0' shape=(?,) dtype=int64>

In [39]:
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")
y

<tf.Tensor 'y:0' shape=(?,) dtype=int64>

In [40]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5

In [41]:
T = tf.one_hot(y, depth=caps2_n_caps, name="T")


In [42]:
with tf.Session():
    print(T.eval(feed_dict={y: np.array([0, 1, 2, 3, 9])}))

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [43]:
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

In [44]:
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 10),
                           name="present_error")

In [45]:
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 10),
                          name="absent_error")

In [46]:
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

In [47]:
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")


In [48]:
mask_with_labels = tf.placeholder_with_default(False, shape=(),
                                               name="mask_with_labels")

In [49]:
reconstruction_targets = tf.cond(mask_with_labels, # condition
                                 lambda: y,        # if True
                                 lambda: y_pred,   # if False
                                 name="reconstruction_targets")

In [50]:
reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=caps2_n_caps,
                                 name="reconstruction_mask")

In [51]:
reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
    name="reconstruction_mask_reshaped")

In [52]:
caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

In [53]:
decoder_input = tf.reshape(caps2_output_masked,
                           [-1, caps2_n_caps * caps2_n_dims],
                           name="decoder_input")

In [54]:
n_hidden1 = 512
n_hidden2 = 1024
n_output = 50 * 50 * 3

In [55]:
with tf.name_scope("decoder"):
    hidden1 = tf.layers.dense(decoder_input, n_hidden1,
                              activation=tf.nn.relu,
                              name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu,
                              name="hidden2")
    decoder_output = tf.layers.dense(hidden2, n_output,
                                     activation=tf.nn.sigmoid,
                                     name="decoder_output")

Instructions for updating:
Use keras.layers.dense instead.


In [56]:
X_flat = tf.reshape(X, [-1, n_output], name="X_flat")
squared_difference = tf.square(X_flat - decoder_output,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                    name="reconstruction_loss")

In [57]:
alpha = 0.0005

loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")

In [58]:
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [59]:
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [60]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [61]:
y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()

In [62]:
test_prec = precision_m(y,y_pred)
test_recall =recall_m(y,y_pred)
test_f1 = f1_m(y,y_pred)

In [63]:
n_epochs = 10
batch_size = 20
restore_checkpoint = True

trainLen = len(X_train)
valLen = len(X_val)


n_iterations_per_epoch = trainLen // batch_size
n_iterations_validation = valLen // batch_size
best_loss_val = np.infty
checkpoint_path = "./my_capsule_network"
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

with tf.Session(config=config) as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()

    for epoch in range(n_epochs):
        start = 0
        end = batch_size
        for iteration in range(1, n_iterations_per_epoch + 1):
            X_batch, y_batch = X_train[start:end],y_train[start:end]
            # Run the training operation and measure the loss:
            _, loss_train = sess.run(
                [training_op, loss],
                feed_dict={X: X_batch.reshape([-1, 50, 50, 3]),
                           y: y_batch,
                           mask_with_labels: True})
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                      iteration, n_iterations_per_epoch,
                      iteration * 100 / n_iterations_per_epoch,
                      loss_train),
                  end="")
            start = start+batch_size
            end = end+batch_size

        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        prec_vals = []
        recall_vals = []
        f1_vals = []
        start = 0
        end = batch_size
        for iteration in range(1, n_iterations_validation + 1):
            X_batch, y_batch = X_val[start:end],y_val[start:end]
            loss_val, acc_val,prec_test,recall_test,f1_m_test = sess.run(
                    [loss, accuracy,test_prec,test_recall,test_f1],
                    feed_dict={X: X_batch.reshape([-1,50, 50, 3]),
                               y: y_batch})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            prec_vals.append(prec_test)
            recall_vals.append(recall_test)
            f1_vals.append(f1_m_test)
            print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                      iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation),
                  end=" " * 10)
            start = start+batch_size
            end = end+batch_size
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        prec_val = np.mean(prec_vals)
        recall_val = np.mean(recall_vals)
        f1_val = np.mean(f1_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f} Precision:{}% Recall:{}% F1_measure:{}% {}".format(
            epoch + 1, acc_val * 100, loss_val, prec_val*100, recall_val*100, f1_val*100,
            " (improved)" if loss_val < best_loss_val else ""))

        # And save the model if it improved:
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val
        

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./my_capsule_network
Epoch: 1  Val accuracy: 97.3810%  Loss: 0.023502 Precision:96.4102564102564% Recall:98.61111111111113% F1_measure:97.34112950360088%  (improved)
Epoch: 2  Val accuracy: 98.8095%  Loss: 0.016654 Precision:98.77344877344876% Recall:98.61111111111113% F1_measure:98.60880320507648%  (improved)
Epoch: 3  Val accuracy: 98.8095%  Loss: 0.017093 Precision:98.77344877344876% Recall:98.61111111111113% F1_measure:98.60880320507648% 
Epoch: 4  Val accuracy: 98.8095%  Loss: 0.016671 Precision:98.77344877344876% Recall:98.61111111111113% F1_measure:98.60880320507648% 
Epoch: 5  Val accuracy: 98.5714%  Loss: 0.016984 Precision:98.73737373737373% Recall:98.17821067821068% F1_measure:98.3829421096502% 
Epoch: 6  Val accuracy: 98.5714%  Loss: 0.018756 Precision:98.37662337662339% Recall:98.61111111111113% F1_measure:98.40176386760237% 
Epoch: 7  Val accura

In [ ]:
testLen = len(X_test)

n_iterations_test = testLen // batch_size
start = 0
end = batch_size

with tf.Session(config=config) as sess:
    saver.restore(sess, checkpoint_path)

    loss_tests = []
    acc_tests = []
    for iteration in range(1, n_iterations_test + 1):
        X_batch, y_batch = X_test[start:end],y_test[start:end]
        loss_test, acc_test = sess.run(
                [loss, accuracy],
                feed_dict={X: X_batch.reshape([-1, 50, 50, 3]),
                           y: y_batch})
        loss_tests.append(loss_test)
        acc_tests.append(acc_test)
        print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                  iteration, n_iterations_test,
                  iteration * 100 / n_iterations_test),
              end=" " * 10)
        start = start+batch_size
        end = end+batch_size
    print(acc_tests)
    loss_test = np.mean(loss_tests)
    acc_test = np.mean(acc_tests)
    print("\rFinal test accuracy: {:.4f}%  Loss: {:.6f}".format(
        acc_test * 100, loss_test))

In [ ]:
n_samples = 1

img = cv2.imread("sample_image.jpg",cv2.IMREAD_COLOR) 
sample_images = cv2.resize(img,(img_width,img_height))
sample_images = sample_images.reshape([-1, 50, 50, 3])
sample_images = sample_images/255
print(sample_images)
checkpoint_path = "./my_capsule_network"
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

with tf.Session(config=config) as sess:
    saver.restore(sess, checkpoint_path)
    caps2_output_value, decoder_output_value, y_pred_value = sess.run(
            [caps2_output, decoder_output, y_pred],
            feed_dict={X: sample_images,
                       y: np.array([], dtype=np.int64)})

In [ ]:

sample_images = sample_images.reshape(-1, 50, 50,3)
reconstructions = decoder_output_value.reshape([-1, 50, 50,3])

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    plt.imshow(sample_images[index], cmap="binary")
    plt.title("Label:" + str(X_test[index]))
    plt.axis("off")

plt.show()

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    plt.title("Predicted:" + str(y_pred_value[index]))
    plt.imshow(reconstructions[index], cmap="binary")
    plt.axis("off")
    
plt.show()

In [ ]:
caps2_output_value.shape


In [ ]:
def tweak_pose_parameters(output_vectors, min=-0.5, max=0.5, n_steps=11):
    steps = np.linspace(min, max, n_steps) # -0.25, -0.15, ..., +0.25
    pose_parameters = np.arange(caps2_n_dims) # 0, 1, ..., 15
    tweaks = np.zeros([caps2_n_dims, n_steps, 1, 1, 1, caps2_n_dims, 1])
    tweaks[pose_parameters, :, 0, 0, 0, pose_parameters, 0] = steps
    output_vectors_expanded = output_vectors[np.newaxis, np.newaxis]
    return tweaks + output_vectors_expanded

In [ ]:
n_steps = 11

tweaked_vectors = tweak_pose_parameters(caps2_output_value, n_steps=n_steps)
tweaked_vectors_reshaped = tweaked_vectors.reshape(
    [-1, 1, caps2_n_caps, caps2_n_dims, 1])

In [ ]:
tweak_labels = np.tile(mnist.test.labels[:n_samples], caps2_n_dims * n_steps)

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    decoder_output_value = sess.run(
            decoder_output,
            feed_dict={caps2_output: tweaked_vectors_reshaped,
                       mask_with_labels: True,
                       y: tweak_labels})

In [ ]:
tweak_reconstructions = decoder_output_value.reshape(
        [caps2_n_dims, n_steps, n_samples, 50, 50])

In [ ]:
for dim in range(3):
    print("Tweaking output dimension #{}".format(dim))
    plt.figure(figsize=(n_steps / 1.2, n_samples / 1.5))
    for row in range(n_samples):
        for col in range(n_steps):
            plt.subplot(n_samples, n_steps, row * n_steps + col + 1)
            plt.imshow(tweak_reconstructions[dim, col, row], cmap="binary")
            plt.axis("off")
    plt.show()